In [1]:
from openimages.download import download_dataset
import os
import shutil
import yaml
from sklearn.model_selection import train_test_split

CLASSES = [
    "Bowl", "Coffee cup", "Mug", "Plate",
    "Spoon", "Fork", "Knife", "Frying pan", "Kitchen knife", "Cutting board", "Ladle", "Wok",
]

TEMP_DIR = "data/openimages_temp"
YOLO_DIR = "data/yolo_dataset"

print("Downloading dataset...")
os.makedirs(TEMP_DIR, exist_ok=True)
download_dataset(TEMP_DIR, CLASSES, annotation_format="darknet", limit=200)

print("Organizing for YOLO...")
os.makedirs(f"{YOLO_DIR}/images/train", exist_ok=True)
os.makedirs(f"{YOLO_DIR}/images/val", exist_ok=True)
os.makedirs(f"{YOLO_DIR}/labels/train", exist_ok=True)
os.makedirs(f"{YOLO_DIR}/labels/val", exist_ok=True)

all_images = []
all_labels = []

for class_name in CLASSES:
    possible_names = [class_name, class_name.lower(), class_name.lower().replace(" ", "_")]
    class_folder = next((os.path.join(TEMP_DIR, name) for name in possible_names if os.path.exists(os.path.join(TEMP_DIR, name))), None)
    
    if not class_folder:
        continue
    
    images_dir = os.path.join(class_folder, "images")
    labels_dir = os.path.join(class_folder, "darknet")
    
    if not os.path.exists(images_dir):
        continue
    
    for img_file in os.listdir(images_dir):
        if img_file.endswith('.jpg'):
            img_path = os.path.join(images_dir, img_file)
            label_path = os.path.join(labels_dir, img_file.replace('.jpg', '.txt'))
            if os.path.exists(label_path):
                all_images.append(img_path)
                all_labels.append(label_path)

train_images, val_images, train_labels, val_labels = train_test_split(
    all_images, all_labels, test_size=0.2, random_state=42
)

for img_path, label_path in zip(train_images, train_labels):
    shutil.copy(img_path, f"{YOLO_DIR}/images/train/{os.path.basename(img_path)}")
    shutil.copy(label_path, f"{YOLO_DIR}/labels/train/{os.path.basename(label_path)}")

for img_path, label_path in zip(val_images, val_labels):
    shutil.copy(img_path, f"{YOLO_DIR}/images/val/{os.path.basename(img_path)}")
    shutil.copy(label_path, f"{YOLO_DIR}/labels/val/{os.path.basename(label_path)}")

data_yaml = {
    "path": os.path.abspath(YOLO_DIR),
    "train": "images/train",
    "val": "images/val",
    "names": {i: name for i, name in enumerate(CLASSES)},
}

yaml_path = os.path.join(YOLO_DIR, "data.yaml")
with open(yaml_path, 'w') as f:
    yaml.dump(data_yaml, f, default_flow_style=False)

print(f"Dataset ready: {len(train_images)} train, {len(val_images)} val")

2025-11-07  14:33:00 INFO Downloading 200 train images for class 'bowl'
100%|██████████| 200/200 [00:07<00:00, 26.95it/s]
2025-11-07  14:33:07 INFO Creating 200 train annotations (darknet) for class 'bowl'
100%|██████████| 200/200 [00:00<00:00, 308.19it/s]
2025-11-07  14:33:08 INFO Downloading 200 train images for class 'coffee cup'
100%|██████████| 200/200 [00:07<00:00, 28.03it/s]
2025-11-07  14:33:15 INFO Creating 200 train annotations (darknet) for class 'coffee cup'
100%|██████████| 200/200 [00:00<00:00, 205.20it/s]
2025-11-07  14:33:17 INFO Downloading 200 train images for class 'mug'
100%|██████████| 200/200 [00:06<00:00, 28.69it/s]
2025-11-07  14:33:24 INFO Creating 200 train annotations (darknet) for class 'mug'
100%|██████████| 200/200 [00:00<00:00, 323.45it/s]
2025-11-07  14:33:24 INFO Downloading 200 train images for class 'plate'
100%|██████████| 200/200 [00:07<00:00, 26.56it/s]
2025-11-07  14:33:32 INFO Creating 200 train annotations (darknet) for class 'plate'
100%|██████

Organizing for YOLO...
Dataset ready: 1484 train, 371 val


In [2]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')

results = model.train(
    data="data/yolo_dataset/data.yaml",
    epochs=50,
    imgsz=640,
    batch=16,
    name='sink_dishes_detector',
    patience=10,
    device='mps'
)

print(f"Best model: runs/detect/sink_dishes_detector/weights/best.pt")

New https://pypi.org/project/ultralytics/8.3.226 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.221 🚀 Python-3.11.12 torch-2.9.0 MPS (Apple M2)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data/yolo_dataset/data.yaml, degrees=0.0, deterministic=True, device=mps, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=sink_dishes_detector3, nbs=64, nms=False, opset=None, opt

KeyboardInterrupt: 